In [1]:
#import
import pandas_datareader.data as web
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import itertools
import math
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

C:\Users\chaeyoung\anaconda3\envs\tf\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


### 분석에 필요한 함수, symbol 등

In [2]:
# 15개 symbols
symbols = ["AMZN","WMT","NVDA","TSLA","PG",
           "NKE","AAPL","JNJ",'GOOGL',"MSFT","PFE",
           "BRK-A","FB","MRK","V"]

In [183]:
# 15개 symbols 수익률
df = web.get_data_yahoo(symbols, start = '2017-08-01', end = '2020-09-01')['Adj Close']
df = df.resample('M').agg('last').pct_change().dropna()
df.index = df.index.strftime('%Y-%m-01')

In [4]:
# random forest
randomforest = pd.read_csv('feature_selection/random_forest_feature_selection.csv')

In [5]:
# adaboost
adaboost = pd.read_csv('feature_selection/adaboost_feature_selection.csv')

In [6]:
# fred 138 factor
fred = pd.read_csv('fred138.csv')

In [7]:
# text mining
google_master = pd.read_csv("textmining_result/google_master.csv")
seeking_master = pd.read_csv("textmining_result/seeking_master.csv")

company_mapping = {'ALPHABET' : 'GOOGL',
 'AMAZON':'AMZN', 'APPLE':'AAPL', 'BERKSHIRE HATHAWAY':'BRK-A',
 'FACEBOOK':'FB', 'JOHNSON & JOHNSON':'JNJ', 'MERCK AND CO':'MRK',
 'MICROSOFT':'MSFT', 'NIKE':'NKE', 'NVIDIA':'NVDA','P&G' :'PG',
 'PFIZER' : 'PFE', 'TESLA':'TSLA', 'VISA':'V', 'WALMART':'WMT'}

google_master['Company'] = google_master.Company.map(company_mapping)

textmining = pd.merge(google_master, seeking_master, on = ['Company', 'Date']) # x : google, y : seeking

textmining.columns = ['Company', 'Date', 'mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',
       'pos_s', 'neg_s', 'neu_s']

textmining_new = textmining.copy()

#textmining_new['pos_g_r'] = textmining['pos_g'] / (textmining['pos_g'] + textmining['neg_g'])
textmining_new['neg_g_r'] = textmining['neg_g'] / (textmining['pos_g'] + textmining['neg_g'])

#textmining_new['pos_s_r'] = textmining['pos_s'] / (textmining['pos_s'] + textmining['neg_s'])
textmining_new['neg_s_r'] = textmining['neg_s'] / (textmining['pos_s'] + textmining['neg_s'])

textmining = textmining_new

textmining = textmining.fillna(0)

In [8]:
# forward selection
def forward(X, y, predictors):
    # 데이터 변수들이 미리정의된 predictors에 있는지 없는지 확인 및 분류
    # 남아있는 변수들 중에 어떤 변수들을 쓰면 가장 낮은 AIC를 도출할 수 있는지 찾는 코드
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    results = [] #results 빈 리스트 선언
    for p in remaining_predictors:
        #x와 y그리고 p 번재 predictor, 상수항 추가해서 regression 돌리기 & results에 append하기
        results.append(processSubset(X=X, y= y, feature_set=predictors+[p]+[]))
    # 데이터프레임으로 변환
    models = pd.DataFrame(results)

    # AIC가 가장 낮은 것을 선택
    best_model = models.loc[models['AIC'].argmin()] # index
    print(models['AIC'])
    print("Processed ", models.shape[0], "models on", len(predictors)+1)
    print('Selected predictors:',best_model['model'].model.exog_names,' AIC:',best_model[1] )
    return best_model


#### 전진선택법 모델

def forward_model(X,y):
    Fmodels = pd.DataFrame(columns=["AIC", "model"])

    # 미리 정의된 데이터 변수
    predictors = []
    # 변수 1~10개 : 0~9 -> 1~10
    for i in range(1, len(X.columns) + 1):
        Forward_result = forward(X=X,y=y,predictors=predictors)
        if i > 1:
            if Forward_result['AIC'] > Fmodel_before:
                break
        Fmodels.loc[i] = Forward_result
        predictors = Fmodels.loc[i]["model"].model.exog_names
        Fmodel_before = Fmodels.loc[i]["AIC"]
        predictors = [ k for k in predictors if k != 'const']

    return(Fmodels['model'][len(Fmodels['model'])])

def processSubset(X,y, feature_set):
    model = sm.OLS(y,sm.add_constant(X[list(feature_set)])) # Modeling
    regr = model.fit() # 모델 학습
    AIC = regr.aic # 모델의 AIC
    return {"model":regr, "AIC":AIC}

In [23]:
# ind5
ind5 = web.DataReader('5_industry_Portfolios', 'famafrench', start = '2017-09-01', end = '2020-09-01')
ind5 = ind5[0] / 100
ind5.index = ind5.index.strftime('%Y-%m-01')
ind5.columns = ['Cnsmr', 'Manuf', 'HiTec', 'Hlth', 'Other']

# 개별 주식 별 분석

### 1. fama3, rf10, ada10 -> regression

* 각 종목별로 pvalues, params, rsquared, rsquared_adj 기록

In [15]:
# fama3
ff3 = web.DataReader('F-F_Research_Data_Factors', 'famafrench',start = '2017-09-01', end = '2020-09-01')
ff3 = ff3[0]/100
ff3.index = ff3.index.strftime('%Y-%m-01')
fama = pd.merge(ff3.reset_index(), df.reset_index(), on = 'Date')

In [16]:
fama

,Date,Mkt-RF,SMB,HML,RF,AMZN,WMT,NVDA,TSLA,PG,NKE,AAPL,JNJ,GOOGL,MSFT,PFE,BRK-A,FB,MRK,V
0,2017-09-01,0.0251,0.0447,0.0311,0.0009,-0.019631,0.000897,0.055064,-0.041585,-0.013981,-0.018178,-0.060244,-0.017829,0.019346,-0.003745,0.052476,0.012120,-0.006396,0.009914,0.016615
1,2017-10-01,0.0225,-0.0194,0.0007,0.0009,0.149717,0.117353,0.156849,-0.028056,-0.043889,0.060559,0.096808,0.072302,0.060921,0.116660,-0.017927,0.020856,0.053784,-0.139622,0.045040
2,2017-11-01,0.0312,-0.0059,-0.0006,0.0008,0.064662,0.113618,-0.028821,-0.068410,0.042275,0.098745,0.020278,0.005547,0.003030,0.016984,0.043677,0.039327,-0.015995,0.003267,0.025545
3,2017-12-01,0.0106,-0.0131,0.0027,0.0009,-0.006187,0.026336,-0.035923,0.008095,0.021002,0.038692,-0.015246,0.002799,0.016629,0.016277,-0.001103,0.020926,-0.004064,0.026753,0.012701
4,2018-01-01,0.0558,-0.0310,-0.0124,0.0011,0.240639,0.079494,0.270284,0.137980,-0.053132,0.090647,-0.010636,-0.010950,0.122290,0.110708,0.022639,0.086610,0.059107,0.052959,0.089546
5,2018-02-01,-0.0365,0.0030,-0.0102,0.0011,0.042429,-0.155629,-0.014848,-0.031752,-0.090572,-0.017444,0.068185,-0.054116,-0.066231,-0.008415,-0.010627,-0.040588,-0.045856,-0.084894,-0.008661
6,2018-03-01,-0.0235,0.0405,-0.0014,0.0012,-0.043049,-0.005661,-0.043017,-0.224246,0.009679,-0.005804,-0.058051,-0.013320,-0.060494,-0.026661,-0.022583,-0.035939,-0.103914,0.013396,-0.027005
7,2018-04-01,0.0029,0.0114,0.0046,0.0014,0.082075,-0.005732,-0.028887,0.104347,-0.079094,0.029350,-0.015020,-0.012954,-0.017895,0.024652,0.031558,-0.028251,0.076413,0.080778,0.060692
8,2018-05-01,0.0265,0.0528,-0.0315,0.0014,0.040539,-0.061044,0.122036,-0.031201,0.011473,0.049861,0.135124,-0.047299,0.079935,0.061467,-0.008994,-0.011870,0.115000,0.011211,0.031920
9,2018-06-01,0.0048,0.0118,-0.0227,0.0014,0.043065,0.037679,-0.060629,0.204474,0.066831,0.112849,-0.009418,0.014379,0.026536,-0.002327,0.009741,-0.017967,0.013244,0.027534,0.013235


In [40]:
for i in range(0,len(symbols)):
    Y = fama[symbols[i]]
    X = fama[['Mkt-RF', 'SMB', 'HML', 'RF']]

    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        ff3_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ff3_reg = pd.concat([ff3_reg, temp], axis = 1)
ff3_reg.columns = symbols

In [41]:
ff3_reg

,AMZN,WMT,NVDA,TSLA,PG,NKE,AAPL,JNJ,GOOGL,MSFT,PFE,BRK-A,FB,MRK,V
R^2,0.657051,0.372644,0.536092,0.28615,0.36321,0.524306,0.535218,0.5555,0.69665,0.75589,0.24204,0.751267,0.613453,0.149096,0.754889
adj_R^2,0.615481,0.296601,0.479861,0.199623,0.286024,0.466646,0.47888,0.501621,0.659881,0.726301,0.150166,0.721117,0.566599,0.0459561,0.725179
p-values,"[['Mkt-RF', 'SMB', 'HML', 'RF'], [2.4428244980...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.0006583996...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [9.9656655197...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.0012457788...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.0008131201...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [1.8997121755...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [3.2295665156...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [3.5808546746...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [1.2270317169...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [3.8451748887...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.0037474464...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [1.1948611845...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [3.3080223824...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.0615882808...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [2.2177779570..."
params,"[['Mkt-RF', 'SMB', 'HML', 'RF'], [1.5204960525...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.6323915964...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [1.6994425146...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [2.4578877872...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.5343016599...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.8669289534...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [1.3812708329...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.7921370068...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [1.0895668188...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.9785909264...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.6515838342...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.8752123812...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [1.3345752092...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.3406852552...","[['Mkt-RF', 'SMB', 'HML', 'RF'], [0.9293422077..."


In [42]:
ff3_reg.to_csv('regression 결과/fama1.csv')

In [43]:
# rf10
for i in range(0,len(symbols)):
    new_rf = randomforest[['index',symbols[i]]].sort_values(by = symbols[i], ascending = False).reset_index()
    del new_rf['level_0']
    new_rf[0:10]['index']

    col = ['Date']
    col2 = list(new_rf[0:10]['index'])
    col = col + col2

    fred_rf10 = pd.merge(fred[col], df[[symbols[i]]].reset_index(), on = 'Date')

    # 회귀분석
    Y = fred_rf10[symbols[i]]
    X = fred_rf10[fred_rf10.columns[1:int(len(fred_rf10.columns)-1)]]

    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        rf10_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        rf10_reg = pd.concat([rf10_reg, temp], axis = 1)

rf10_reg.columns = symbols    

In [44]:
rf10_reg

,AMZN,WMT,NVDA,TSLA,PG,NKE,AAPL,JNJ,GOOGL,MSFT,PFE,BRK-A,FB,MRK,V
R^2,0.418699,0.464088,0.320651,0.50659,0.419773,0.383802,0.495063,0.434401,0.558333,0.601975,0.312326,0.630678,0.514954,0.552993,0.666755
adj_R^2,0.203402,0.265603,0.0690401,0.323846,0.204874,0.15558,0.30805,0.22492,0.394752,0.454559,0.0576313,0.493893,0.335307,0.387434,0.54333
p-values,"[['WPSID62', 'HOUSTMW', 'AMDMNOx', 'CE16OV', '...","[['UEMP27OV', 'AMDMNOx', 'TB3SMFFM', 'UMCSENTx...","[['IPFPNSS', 'IPB51222S', 'GS1', 'AMDMNOx', 'H...","[['DTCOLNVHFNM', 'IPB51222S', 'GS10', 'ANDENOx...","[['UMCSENTx', 'EXCAUSx', 'IPB51222S', 'PERMITW...","[['CES1021000001', 'IPB51222S', 'CLAIMSx', 'DN...","[['IPDCONGD', 'HOUSTMW', 'S&P 500', 'CPIAPPSL'...","[['HOUSTW', 'IPDCONGD', 'TWEXAFEGSMTHx', 'IPBU...","[['UEMP15T26', 'VXOCLSx', 'IPDCONGD', 'UMCSENT...","[['WPSID62', 'IPDCONGD', 'VXOCLSx', 'UMCSENTx'...","[['GS10', 'BAA', 'AAA', 'USWTRADE', 'VXOCLSx',...","[['VXOCLSx', 'UEMP27OV', 'UMCSENTx', 'AMDMNOx'...","[['DNDGRG3M086SBEA', 'UEMP5TO14', 'VXOCLSx', '...","[['UMCSENTx', 'IPB51222S', 'HOUSTNE', 'IPDMAT'...","[['IPDCONGD', 'VXOCLSx', 'HOUSTNE', 'USGOVT', ..."
params,"[['WPSID62', 'HOUSTMW', 'AMDMNOx', 'CE16OV', '...","[['UEMP27OV', 'AMDMNOx', 'TB3SMFFM', 'UMCSENTx...","[['IPFPNSS', 'IPB51222S', 'GS1', 'AMDMNOx', 'H...","[['DTCOLNVHFNM', 'IPB51222S', 'GS10', 'ANDENOx...","[['UMCSENTx', 'EXCAUSx', 'IPB51222S', 'PERMITW...","[['CES1021000001', 'IPB51222S', 'CLAIMSx', 'DN...","[['IPDCONGD', 'HOUSTMW', 'S&P 500', 'CPIAPPSL'...","[['HOUSTW', 'IPDCONGD', 'TWEXAFEGSMTHx', 'IPBU...","[['UEMP15T26', 'VXOCLSx', 'IPDCONGD', 'UMCSENT...","[['WPSID62', 'IPDCONGD', 'VXOCLSx', 'UMCSENTx'...","[['GS10', 'BAA', 'AAA', 'USWTRADE', 'VXOCLSx',...","[['VXOCLSx', 'UEMP27OV', 'UMCSENTx', 'AMDMNOx'...","[['DNDGRG3M086SBEA', 'UEMP5TO14', 'VXOCLSx', '...","[['UMCSENTx', 'IPB51222S', 'HOUSTNE', 'IPDMAT'...","[['IPDCONGD', 'VXOCLSx', 'HOUSTNE', 'USGOVT', ..."


In [45]:
rf10_reg.to_csv('regression 결과/rf1.csv')

In [46]:
# ada10
for i in range(0,len(symbols)):
    new_ada = adaboost[['index',symbols[i]]].sort_values(by = symbols[i], ascending = False).reset_index()
    del new_ada['level_0']
    new_ada[0:10]['index']

    col = ['Date']
    col2 = list(new_ada[0:10]['index'])
    col = col + col2

    fred_ada10 = pd.merge(fred[col], df[[symbols[i]]].reset_index(), on = 'Date')

    # 회귀분석
    Y = fred_ada10[symbols[i]]
    X = fred_ada10[fred_ada10.columns[1:int(len(fred_ada10.columns)-1)]]

    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        ada10_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ada10_reg = pd.concat([ada10_reg, temp], axis = 1)

ada10_reg.columns = symbols    

In [47]:
ada10_reg

,AMZN,WMT,NVDA,TSLA,PG,NKE,AAPL,JNJ,GOOGL,MSFT,PFE,BRK-A,FB,MRK,V
R^2,0.400926,0.411128,0.369026,0.579224,0.393151,0.414258,0.588036,0.25752,0.644062,0.621912,0.295598,0.535426,0.366072,0.544915,0.619802
adj_R^2,0.179046,0.193027,0.135333,0.423381,0.168392,0.197317,0.435457,-0.0174728,0.512233,0.48188,0.0347081,0.363361,0.131284,0.376365,0.478987
p-values,"[['HOUSTMW', 'WPSID62', 'WPSFD49207', 'AMDMNOx...","[['UEMP27OV', 'CPIAPPSL', 'T10YFFM', 'AMDMNOx'...","[['GS1', 'CES1021000001', 'AAA', 'EXSZUSx', 'U...","[['DTCOLNVHFNM', 'GS10', 'CUSR0000SAD', 'ANDEN...","[['UMCSENTx', 'EXCAUSx', 'HOUSTW', 'IPB51222S'...","[['UMCSENTx', 'HOUSTNE', 'VXOCLSx', 'EXSZUSx',...","[['IPDCONGD', 'CPIAPPSL', 'UEMP15OV', 'HWIURAT...","[['IPDCONGD', 'HOUSTW', 'TWEXAFEGSMTHx', 'IPB5...","[['UEMP15T26', 'VXOCLSx', 'IPB51222S', 'PERMIT...","[['AMDMNOx', 'IPDCONGD', 'UMCSENTx', 'WPSID62'...","[['GS10', 'IPB51222S', 'ANDENOx', 'AAA', 'REAL...","[['UMCSENTx', 'UEMP27OV', 'CUSR0000SAD', 'TWEX...","[['DNDGRG3M086SBEA', 'UEMP5TO14', 'USWTRADE', ...","[['UMCSENTx', 'IPB51222S', 'HOUSTNE', 'IPFINAL...","[['VXOCLSx', 'HOUSTNE', 'UEMP15T26', 'USFIRE',..."
params,"[['HOUSTMW', 'WPSID62', 'WPSFD49207', 'AMDMNOx...","[['UEMP27OV', 'CPIAPPSL', 'T10YFFM', 'AMDMNOx'...","[['GS1', 'CES1021000001', 'AAA', 'EXSZUSx', 'U...","[['DTCOLNVHFNM', 'GS10', 'CUSR0000SAD', 'ANDEN...","[['UMCSENTx', 'EXCAUSx', 'HOUSTW', 'IPB51222S'...","[['UMCSENTx', 'HOUSTNE', 'VXOCLSx', 'EXSZUSx',...","[['IPDCONGD', 'CPIAPPSL', 'UEMP15OV', 'HWIURAT...","[['IPDCONGD', 'HOUSTW', 'TWEXAFEGSMTHx', 'IPB5...","[['UEMP15T26', 'VXOCLSx', 'IPB51222S', 'PERMIT...","[['AMDMNOx', 'IPDCONGD', 'UMCSENTx', 'WPSID62'...","[['GS10', 'IPB51222S', 'ANDENOx', 'AAA', 'REAL...","[['UMCSENTx', 'UEMP27OV', 'CUSR0000SAD', 'TWEX...","[['DNDGRG3M086SBEA', 'UEMP5TO14', 'USWTRADE', ...","[['UMCSENTx', 'IPB51222S', 'HOUSTNE', 'IPFINAL...","[['VXOCLSx', 'HOUSTNE', 'UEMP15T26', 'USFIRE',..."


In [48]:
ada10_reg.to_csv('regression 결과/ada1.csv')

### 2. fama3, rf10, ada10 + text mining -> regression

In [49]:
# fama
fama_text = pd.merge(textmining, fama, on = 'Date')
fama_text = fama_text.fillna(0)

for i in range(0,len(symbols)):
    fama_text_new = fama_text[fama_text['Company'] == symbols[i]]
    Y = fama_text_new[symbols[i]]
    X = fama_text_new[['Mkt-RF', 'SMB', 'HML',
       'RF','mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',
       'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']]

    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        ff3_text_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ff3_text_reg = pd.concat([ff3_text_reg, temp], axis = 1)
ff3_text_reg.columns = symbols

In [50]:
ff3_text_reg

,AMZN,WMT,NVDA,TSLA,PG,NKE,AAPL,JNJ,GOOGL,MSFT,PFE,BRK-A,FB,MRK,V
R^2,0.741332,0.516689,0.683386,0.618867,0.52079,0.711453,0.69846,0.601252,0.863947,0.819901,0.358925,0.849622,0.704722,0.386017,0.846175
adj_R^2,0.583883,0.2225,0.490665,0.386873,0.229098,0.535816,0.514913,0.358535,0.781132,0.710276,-0.0312938,0.758088,0.524987,0.0122884,0.752543
p-values,"[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos..."
params,"[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos...","[['Mkt-RF', 'SMB', 'HML', 'RF', 'mean_g', 'pos..."


In [51]:
ff3_text_reg.to_csv('regression 결과/fama2.csv')

In [52]:
# rf10
for i in range(0,len(symbols)):
    new_rf = randomforest[['index',symbols[i]]].sort_values(by = symbols[i], ascending = False).reset_index()
    del new_rf['level_0']
    new_rf[0:10]['index']

    col = ['Date']
    col2 = list(new_rf[0:10]['index'])
    col = col + col2

    fred_rf10 = pd.merge(fred[col], df[[symbols[i]]].reset_index(), on = 'Date')
    fred_rf10 = pd.merge(textmining,fred_rf10, on = 'Date')
    
    # 회귀분석
    fred_rf10_new = fred_rf10[fred_rf10['Company'] == symbols[i]]
    Y = fred_rf10_new[symbols[i]]
    X = fred_rf10_new[fred_rf10_new.columns[2:int(len(fred_rf10_new.columns)-1)]]
    
    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        rf10_text_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        rf10_text_reg = pd.concat([rf10_text_reg, temp], axis = 1)

rf10_text_reg.columns = symbols    

In [53]:
rf10_text_reg

,AMZN,WMT,NVDA,TSLA,PG,NKE,AAPL,JNJ,GOOGL,MSFT,PFE,BRK-A,FB,MRK,V
R^2,0.682647,0.576792,0.549718,0.698946,0.553769,0.61103,0.73937,0.691537,0.780878,0.737089,0.428714,0.731553,0.633759,0.670974,0.740483
adj_R^2,0.309291,0.0789005,0.0199747,0.344765,0.0287906,0.153418,0.432747,0.32864,0.523088,0.427782,-0.243388,0.415733,0.202886,0.283884,0.435169
p-values,"[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s..."
params,"[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s..."


In [54]:
rf10_text_reg.to_csv('regression 결과/rf2.csv')

In [55]:
# ada10
for i in range(0,len(symbols)):
    new_ada = adaboost[['index',symbols[i]]].sort_values(by = symbols[i], ascending = False).reset_index()
    del new_ada['level_0']
    new_ada[0:10]['index']

    col = ['Date']
    col2 = list(new_ada[0:10]['index'])
    col = col + col2

    fred_ada10 = pd.merge(fred[col], df[[symbols[i]]].reset_index(), on = 'Date')
    fred_ada10 = pd.merge(textmining,fred_ada10, on = 'Date')
    
    # 회귀분석
    fred_ada10_new = fred_ada10[fred_ada10['Company'] == symbols[i]]
    
    Y = fred_ada10_new[symbols[i]]
    X = fred_ada10_new[fred_ada10_new.columns[2:int(len(fred_ada10_new.columns)-1)]]

    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        ada10_text_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ada10_text_reg = pd.concat([ada10_text_reg, temp], axis = 1)

ada10_text_reg.columns = symbols   

In [56]:
ada10_text_reg

,AMZN,WMT,NVDA,TSLA,PG,NKE,AAPL,JNJ,GOOGL,MSFT,PFE,BRK-A,FB,MRK,V
R^2,0.69231,0.510666,0.567561,0.751739,0.549476,0.655039,0.749192,0.610097,0.808633,0.758124,0.553679,0.723368,0.665294,0.628632,0.728016
adj_R^2,0.330321,-0.065021,0.0588088,0.459668,0.0194473,0.249201,0.454124,0.151388,0.583494,0.473563,0.0285944,0.39792,0.271522,0.191729,0.408035
p-values,"[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s..."
params,"[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s..."


In [57]:
ada10_text_reg.to_csv('regression 결과/ada2.csv')

### 2-2. textmining only

In [67]:
# fama
fama_text = pd.merge(textmining, fama, on = 'Date')
fama_text = fama_text.fillna(0)

for i in range(0,len(symbols)):
    fama_text_new = fama_text[fama_text['Company'] == symbols[i]]
    Y = fama_text_new[symbols[i]]
    X = fama_text_new[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',
       'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']]

    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        ff3_text_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ff3_text_reg = pd.concat([ff3_text_reg, temp], axis = 1)
ff3_text_reg.columns = symbols

In [75]:
ff3_text_reg

,AMZN,WMT,NVDA,TSLA,PG,NKE,AAPL,JNJ,GOOGL,MSFT,PFE,BRK-A,FB,MRK,V
R^2,0.421522,0.228272,0.418125,0.495155,0.33139,0.49452,0.574904,0.37322,0.309194,0.362028,0.1635,0.227687,0.325006,0.250877,0.346398
adj_R^2,0.207271,-0.0575528,0.202616,0.308176,0.083757,0.307305,0.417461,0.141079,0.0533398,0.125742,-0.146314,-0.0583553,0.0750082,-0.0265766,0.104323
p-values,"[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s..."
params,"[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s...","[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s..."


In [68]:
ff3_text_reg.to_csv('regression 결과/textonly.csv')

### 3. textmining + forward selection

In [58]:
# fama
fama_text = pd.merge(textmining, fama, on = 'Date')
fama_text = fama_text.fillna(0)

for i in range(0,len(symbols)):
    fama_text_new = fama_text[fama_text['Company'] == symbols[i]]
    Y = fama_text_new[symbols[i]]
    X = fama_text_new[['Mkt-RF', 'SMB', 'HML',
       'RF','mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',
       'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']]

    model_forward = forward_model(X, Y)
    results = model_forward
    
    if (i == 0):
        ff3_text_forward = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ff3_text_forward = pd.concat([ff3_text_forward, temp], axis = 1)
ff3_text_forward.columns = symbols

0    -91.470002
1    -69.429107
2    -69.283913
3    -74.801946
4    -69.088523
5    -73.814291
6    -72.609742
7    -74.780991
8    -68.844361
9    -70.839211
10   -69.149810
11   -69.123151
12   -69.606808
13   -68.785669
Name: AIC, dtype: float64
Processed  14 models on 1
Selected predictors: ['const', 'Mkt-RF']  AIC: -91.4700018654608
0    -91.389177
1    -98.467441
2    -92.233024
3    -89.514411
4    -97.069394
5    -93.290287
6    -93.952331
7    -89.478532
8    -91.693060
9    -90.077278
10   -89.478826
11   -90.008622
12   -89.612412
Name: AIC, dtype: float64
Processed  13 models on 2
Selected predictors: ['const', 'Mkt-RF', 'HML']  AIC: -98.46744092854237
0     -98.566026
1     -97.322643
2     -96.926102
3    -103.711539
4     -98.402751
5    -101.142149
6     -97.816086
7     -98.930953
8     -96.479131
9     -96.614773
10    -96.493165
11    -98.092281
Name: AIC, dtype: float64
Processed  12 models on 3
Selected predictors: ['const', 'Mkt-RF', 'HML', 'pos_g']  AIC: -103.71

0    -114.805924
1    -114.431959
2    -114.607306
3    -119.314316
4    -119.402389
5    -116.343486
6    -114.478907
7    -117.049873
8    -114.429776
9    -115.841568
10   -115.313934
11   -118.462860
12   -114.840584
Name: AIC, dtype: float64
Processed  13 models on 2
Selected predictors: ['const', 'Mkt-RF', 'pos_g']  AIC: -119.40238875304144
0    -118.372795
1    -117.493850
2    -117.602318
3    -118.216596
4    -118.151379
5    -118.448554
6    -119.796654
7    -117.494418
8    -118.661830
9    -117.942144
10   -118.044798
11   -117.743825
Name: AIC, dtype: float64
Processed  12 models on 3
Selected predictors: ['const', 'Mkt-RF', 'pos_g', 'mean_s']  AIC: -119.79665408929466
0    -118.967494
1    -118.440593
2    -117.860757
3    -117.957601
4    -117.854211
5    -118.005284
6    -121.027753
7    -117.839924
8    -117.938997
9    -117.886334
10   -121.314789
Name: AIC, dtype: float64
Processed  11 models on 4
Selected predictors: ['const', 'Mkt-RF', 'pos_g', 'mean_s', 'neg_s_r']

0    -101.194234
1     -92.571703
2     -93.585589
3     -93.569088
4     -92.592621
5     -92.963651
6     -92.574085
7     -93.829117
8     -92.703754
9     -92.636980
10    -92.409499
11    -93.252992
12    -92.815082
13    -92.823812
Name: AIC, dtype: float64
Processed  14 models on 1
Selected predictors: ['const', 'Mkt-RF']  AIC: -101.1942341725024
0    -100.101077
1     -99.259086
2     -99.279015
3     -99.265909
4     -99.843978
5     -99.255873
6    -104.064679
7     -99.309584
8     -99.205213
9     -99.214702
10    -99.688058
11    -99.271913
12    -99.235430
Name: AIC, dtype: float64
Processed  13 models on 2
Selected predictors: ['const', 'Mkt-RF', 'neu_g']  AIC: -104.06467899295387
0    -103.029375
1    -102.085288
2    -102.078026
3    -102.086083
4    -102.086604
5    -102.107674
6    -102.152311
7    -102.128349
8    -102.110971
9    -102.694484
10   -102.096625
11   -102.235223
Name: AIC, dtype: float64
Processed  12 models on 3
Selected predictors: ['const', 'Mkt-RF'

In [59]:
ff3_text_forward

,AMZN,WMT,NVDA,TSLA,PG,NKE,AAPL,JNJ,GOOGL,MSFT,PFE,BRK-A,FB,MRK,V
R^2,0.670757,0.407251,0.6083,0.451849,0.416954,0.646637,0.587821,0.545789,0.802311,0.710044,0.308632,0.756,0.573059,0.2484,0.731439
adj_R^2,0.629602,0.353365,0.559338,0.402017,0.322914,0.575964,0.55035,0.519071,0.762773,0.673799,0.267964,0.733818,0.560861,0.180073,0.707024
p-values,"[['const', 'Mkt-RF', 'HML', 'pos_g', 'pos_s'],...","[['const', 'Mkt-RF', 'SMB', 'pos_s'], [0.28264...","[['const', 'Mkt-RF', 'HML', 'neg_g', 'neg_s_r'...","[['const', 'mean_s', 'Mkt-RF', 'pos_g'], [0.39...","[['const', 'Mkt-RF', 'pos_s', 'SMB', 'neu_s', ...","[['const', 'Mkt-RF', 'pos_g', 'mean_s', 'neg_s...","[['const', 'Mkt-RF', 'mean_s', 'HML'], [0.0027...","[['const', 'Mkt-RF', 'SMB'], [0.67238981145916...","[['const', 'Mkt-RF', 'mean_g', 'SMB', 'neu_g',...","[['const', 'Mkt-RF', 'HML', 'mean_g', 'neg_s_r...","[['const', 'Mkt-RF', 'neu_g'], [0.039170187726...","[['const', 'Mkt-RF', 'SMB', 'neg_g'], [0.10870...","[['const', 'Mkt-RF'], [0.6300107933413512, 5.8...","[['const', 'mean_g', 'Mkt-RF', 'pos_s'], [0.64...","[['const', 'Mkt-RF', 'HML', 'pos_g'], [0.18599..."
params,"[['const', 'Mkt-RF', 'HML', 'pos_g', 'pos_s'],...","[['const', 'Mkt-RF', 'SMB', 'pos_s'], [-0.0151...","[['const', 'Mkt-RF', 'HML', 'neg_g', 'neg_s_r'...","[['const', 'mean_s', 'Mkt-RF', 'pos_g'], [0.08...","[['const', 'Mkt-RF', 'pos_s', 'SMB', 'neu_s', ...","[['const', 'Mkt-RF', 'pos_g', 'mean_s', 'neg_s...","[['const', 'Mkt-RF', 'mean_s', 'HML'], [0.0625...","[['const', 'Mkt-RF', 'SMB'], [-0.0026179120787...","[['const', 'Mkt-RF', 'mean_g', 'SMB', 'neu_g',...","[['const', 'Mkt-RF', 'HML', 'mean_g', 'neg_s_r...","[['const', 'Mkt-RF', 'neu_g'], [-0.14604494291...","[['const', 'Mkt-RF', 'SMB', 'neg_g'], [0.01703...","[['const', 'Mkt-RF'], [0.0049861083265404366, ...","[['const', 'mean_g', 'Mkt-RF', 'pos_s'], [-0.0...","[['const', 'Mkt-RF', 'HML', 'pos_g'], [-0.0220..."


In [60]:
ff3_text_forward.to_csv('regression 결과/fama3.csv')

In [61]:
# rf10
for i in range(0,len(symbols)):
    new_rf = randomforest[['index',symbols[i]]].sort_values(by = symbols[i], ascending = False).reset_index()
    del new_rf['level_0']
    new_rf[0:10]['index']

    col = ['Date']
    col2 = list(new_rf[0:10]['index'])
    col = col + col2

    fred_rf10 = pd.merge(fred[col], df[[symbols[i]]].reset_index(), on = 'Date')
    fred_rf10 = pd.merge(textmining,fred_rf10, on = 'Date')
    
    # 회귀분석
    fred_rf10_new = fred_rf10[fred_rf10['Company'] == symbols[i]]
    Y = fred_rf10_new[symbols[i]]
    X = fred_rf10_new[fred_rf10_new.columns[2:int(len(fred_rf10_new.columns)-1)]]

    model_forward = forward_model(X, Y)
    results = model_forward
    
    if (i == 0):
        rf10_text_forward = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        rf10_text_forward = pd.concat([rf10_text_forward, temp], axis = 1)

rf10_text_forward.columns = symbols    

0    -69.088523
1    -73.814291
2    -72.609742
3    -74.780991
4    -68.844361
5    -70.839211
6    -69.149810
7    -69.123151
8    -69.606808
9    -68.785669
10   -72.281701
11   -68.933941
12   -75.168998
13   -76.145590
14   -77.014178
15   -74.479415
16   -72.141591
17   -69.099091
18   -72.981783
19   -72.220730
Name: AIC, dtype: float64
Processed  20 models on 1
Selected predictors: ['const', 'WPSFD49207']  AIC: -77.01417787850522
0    -75.441490
1    -79.131831
2    -77.799304
3    -79.780510
4    -75.047221
5    -76.302644
6    -75.201382
7    -76.280276
8    -75.492183
9    -75.365141
10   -75.907315
11   -75.026215
12   -76.165008
13   -76.127323
14   -75.132681
15   -75.016001
16   -75.020765
17   -75.190701
18   -76.558799
Name: AIC, dtype: float64
Processed  19 models on 2
Selected predictors: ['const', 'WPSFD49207', 'neu_g']  AIC: -79.78051036842186
0    -77.862572
1    -78.068373
2    -77.981439
3    -77.782506
4    -79.480108
5    -77.785761
6    -78.817541
7    -78.03

0    -116.687200
1    -116.800274
2    -116.186594
3    -115.975375
4    -118.868145
5    -120.598824
6    -117.823600
7    -116.185194
8    -116.387046
9    -117.800615
10   -119.477071
11   -119.344973
12   -117.390748
13   -119.782719
14   -115.790850
15   -116.571745
16   -119.164239
17   -120.885017
18   -116.105798
19   -117.057402
Name: AIC, dtype: float64
Processed  20 models on 1
Selected predictors: ['const', 'HOUSTW']  AIC: -120.8850165198835
0    -119.581186
1    -119.273391
2    -119.220091
3    -118.975340
4    -120.919521
5    -121.093041
6    -122.366318
7    -119.701700
8    -119.371886
9    -119.827545
10   -120.307621
11   -119.814039
12   -119.472381
13   -119.450498
14   -120.354733
15   -119.007022
16   -119.869985
17   -119.115729
18   -118.901338
Name: AIC, dtype: float64
Processed  19 models on 2
Selected predictors: ['const', 'HOUSTW', 'neg_s']  AIC: -122.3663182422637
0    -120.558226
1    -120.423217
2    -120.414767
3    -120.458416
4    -120.627036
5    -1

0    -115.442492
1    -115.326416
2    -115.556330
3    -116.339988
4    -115.295672
5    -120.850323
6    -115.269594
7    -116.547413
8    -115.749042
9    -116.679119
10   -115.478300
11   -115.640124
12   -115.695451
13   -115.833640
14   -115.281488
15   -116.159708
16   -115.470676
17   -115.372654
Name: AIC, dtype: float64
Processed  18 models on 3
Selected predictors: ['const', 'pos_g', 'neu_s', 'neg_s']  AIC: -120.85032333976497
0    -119.005149
1    -118.902967
2    -119.162337
3    -118.851347
4    -118.893541
5    -119.023544
6    -119.075412
7    -119.127676
8    -119.538795
9    -118.932669
10   -118.878268
11   -118.928732
12   -118.905646
13   -118.940886
14   -119.944368
15   -119.026913
16   -118.852080
Name: AIC, dtype: float64
Processed  17 models on 4
Selected predictors: ['const', 'pos_g', 'neu_s', 'neg_s', 'PPICMM']  AIC: -119.94436779395775
0    -102.397722
1     -96.501572
2    -103.048475
3     -96.295141
4     -96.281284
5     -95.396601
6     -97.084726
7   

0    -129.909077
1    -130.378431
2    -131.632074
3    -129.897814
4    -130.546392
5    -130.824592
6    -130.385974
7    -130.062706
8    -130.347446
9    -130.577708
10   -129.769929
11   -129.805928
12   -129.989833
13   -130.547056
14   -130.284568
15   -129.754463
16   -133.878974
17   -130.484647
Name: AIC, dtype: float64
Processed  18 models on 3
Selected predictors: ['const', 'VXOCLSx', 'UMCSENTx', 'USGOVT']  AIC: -133.87897418512762
0    -132.369494
1    -132.942284
2    -133.520488
3    -131.879093
4    -133.580767
5    -133.088303
6    -132.635971
7    -132.204738
8    -132.922080
9    -132.648175
10   -131.975567
11   -132.566217
12   -131.883628
13   -132.396346
14   -133.146713
15   -134.180801
16   -132.239130
Name: AIC, dtype: float64
Processed  17 models on 4
Selected predictors: ['const', 'VXOCLSx', 'UMCSENTx', 'USGOVT', 'USFIRE']  AIC: -134.1808014131337
0    -132.382403
1    -132.805619
2    -133.441916
3    -132.183961
4    -134.581278
5    -133.325835
6    -133.

In [62]:
rf10_text_forward

,AMZN,WMT,NVDA,TSLA,PG,NKE,AAPL,JNJ,GOOGL,MSFT,PFE,BRK-A,FB,MRK,V
R^2,0.335582,0.274039,0.440579,0.554281,0.513958,0.375794,0.546584,0.313753,0.661665,0.559567,0.224325,0.61395,0.173854,0.292207,0.595894
adj_R^2,0.27518,0.183294,0.370652,0.498566,0.375089,0.319048,0.505365,0.251367,0.619373,0.471481,0.178697,0.536739,0.15025,0.250572,0.530716
p-values,"[['const', 'WPSFD49207', 'neu_g', 'ANDENOx'], ...","[['const', 'UMCSENTx', 'TB3SMFFM', 'UEMP27OV',...","[['const', 'AAA', 'neg_g', 'neg_s_r', 'mean_s'...","[['const', 'mean_s', 'S&P 500', 'neu_s', 'PERM...","[['const', 'HOUSTW', 'neg_s', 'pos_s', 'neg_s_...","[['const', 'neg_s', 'pos_g', 'DNDGRG3M086SBEA'...","[['const', 'mean_s', 'UMCSENTx', 'neu_s'], [0....","[['const', 'pos_g', 'neu_s', 'neg_s'], [0.0238...","[['const', 'neg_g_r', 'UMCSENTx', 'VXOCLSx', '...","[['const', 'AMDMNOx', 'VXOCLSx', 'neg_s', 'PER...","[['const', 'CONSPI', 'neu_g'], [0.011410400046...","[['const', 'VXOCLSx', 'UMCSENTx', 'USGOVT', 'U...","[['const', 'USWTRADE'], [0.00963001212797248, ...","[['const', 'IPB51222S', 'mean_g'], [0.00748328...","[['const', 'PERMITW', 'VXOCLSx', 'IPDCONGD', '..."
params,"[['const', 'WPSFD49207', 'neu_g', 'ANDENOx'], ...","[['const', 'UMCSENTx', 'TB3SMFFM', 'UEMP27OV',...","[['const', 'AAA', 'neg_g', 'neg_s_r', 'mean_s'...","[['const', 'mean_s', 'S&P 500', 'neu_s', 'PERM...","[['const', 'HOUSTW', 'neg_s', 'pos_s', 'neg_s_...","[['const', 'neg_s', 'pos_g', 'DNDGRG3M086SBEA'...","[['const', 'mean_s', 'UMCSENTx', 'neu_s'], [0....","[['const', 'pos_g', 'neu_s', 'neg_s'], [-0.086...","[['const', 'neg_g_r', 'UMCSENTx', 'VXOCLSx', '...","[['const', 'AMDMNOx', 'VXOCLSx', 'neg_s', 'PER...","[['const', 'CONSPI', 'neu_g'], [1.366682233970...","[['const', 'VXOCLSx', 'UMCSENTx', 'USGOVT', 'U...","[['const', 'USWTRADE'], [2.0045956896129407, -...","[['const', 'IPB51222S', 'mean_g'], [-0.4068359...","[['const', 'PERMITW', 'VXOCLSx', 'IPDCONGD', '..."


In [63]:
rf10_text_forward.to_csv('regression 결과/rf3.csv')

In [64]:
# ada10
for i in range(0,len(symbols)):
    new_ada = adaboost[['index',symbols[i]]].sort_values(by = symbols[i], ascending = False).reset_index()
    del new_ada['level_0']
    new_ada[0:10]['index']

    col = ['Date']
    col2 = list(new_ada[0:10]['index'])
    col = col + col2

    fred_ada10 = pd.merge(fred[col], df[[symbols[i]]].reset_index(), on = 'Date')
    fred_ada10 = pd.merge(textmining,fred_ada10, on = 'Date')
    
    # 회귀분석
    fred_ada10_new = fred_ada10[fred_ada10['Company'] == symbols[i]]
    
    Y = fred_ada10_new[symbols[i]]
    X = fred_ada10_new[fred_ada10_new.columns[2:int(len(fred_ada10_new.columns)-1)]]

    model_forward = forward_model(X, Y)
    results = model_forward
    
    if (i == 0):
        ada10_text_forward = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ada10_text_forward = pd.concat([ada10_text_forward, temp], axis = 1)

ada10_text_forward.columns = symbols   

0    -69.088523
1    -73.814291
2    -72.609742
3    -74.780991
4    -68.844361
5    -70.839211
6    -69.149810
7    -69.123151
8    -69.606808
9    -68.785669
10   -68.933941
11   -72.281701
12   -77.014178
13   -75.168998
14   -71.173040
15   -77.376949
16   -73.141739
17   -69.911100
18   -68.896512
19   -74.987110
Name: AIC, dtype: float64
Processed  20 models on 1
Selected predictors: ['const', 'CLF16OV']  AIC: -77.37694948856239
0    -75.727890
1    -79.642967
2    -78.617801
3    -80.109162
4    -75.439273
5    -75.883214
6    -76.056959
7    -77.047064
8    -76.150273
9    -75.859281
10   -75.522673
11   -75.836736
12   -76.006555
13   -76.150878
14   -76.067629
15   -75.385075
16   -75.389817
17   -79.742404
18   -75.435727
Name: AIC, dtype: float64
Processed  19 models on 2
Selected predictors: ['const', 'CLF16OV', 'neu_g']  AIC: -80.10916169408895
0    -78.162154
1    -78.471626
2    -78.485768
3    -78.109538
4    -78.946825
5    -78.293149
6    -79.491070
7    -78.575288
8

0    -121.805296
1    -121.668318
2    -121.721778
3    -121.616120
4    -121.920433
5    -121.690870
6    -121.700209
7    -124.118406
8    -121.789194
9    -121.682307
10   -121.745896
11   -121.731622
12   -122.329431
13   -122.023101
14   -121.864872
15   -122.203258
16   -122.323427
Name: AIC, dtype: float64
Processed  17 models on 4
Selected predictors: ['const', 'HOUSTW', 'neg_s', 'pos_s', 'neg_s_r']  AIC: -124.11840615852458
0    -122.472636
1    -122.420747
2    -122.274054
3    -122.181804
4    -122.151199
5    -122.288208
6    -122.297237
7    -122.127818
8    -122.122173
9    -122.205320
10   -122.408037
11   -123.608130
12   -123.902792
13   -122.297460
14   -123.981158
15   -123.479774
Name: AIC, dtype: float64
Processed  16 models on 5
Selected predictors: ['const', 'HOUSTW', 'neg_s', 'pos_s', 'neg_s_r', 'PERMITNE']  AIC: -123.98115757764077
0     -95.649309
1     -97.130305
2     -93.330258
3     -94.407442
4     -99.929004
5     -93.236758
6    -103.256267
7     -93.72

0    -102.208326
1    -102.555276
2    -102.528164
3    -102.393916
4    -102.250658
5    -102.232281
6    -102.556968
7    -103.152515
8    -102.318806
9    -103.689295
10   -105.053822
11   -105.201171
12   -102.253198
13   -105.342450
14   -106.436981
15   -105.064307
16   -105.563066
17   -107.066402
18   -102.209143
Name: AIC, dtype: float64
Processed  19 models on 2
Selected predictors: ['const', 'neg_g_r', 'UMCSENTx']  AIC: -107.06640166164112
0    -105.166265
1    -106.036442
2    -106.097555
3    -106.073892
4    -105.093594
5    -106.014558
6    -106.162425
7    -105.080945
8    -105.066436
9    -105.135643
10   -121.559466
11   -106.291885
12   -105.072031
13   -105.509009
14   -105.315656
15   -105.230748
16   -105.139000
17   -105.597751
Name: AIC, dtype: float64
Processed  18 models on 3
Selected predictors: ['const', 'neg_g_r', 'UMCSENTx', 'VXOCLSx']  AIC: -121.55946555652253
0    -120.111237
1    -120.049347
2    -119.713470
3    -119.705004
4    -120.533366
5    -120.7

0    -121.424754
1    -118.291345
2    -120.354038
3    -118.290235
4    -119.903747
5    -120.187524
6    -118.575289
7    -118.490525
8    -119.792480
9    -119.528182
10   -118.225466
11   -120.009106
12   -119.095052
13   -120.884216
14   -119.459259
15   -118.222730
16   -118.277162
17   -119.131584
18   -118.336069
Name: AIC, dtype: float64
Processed  19 models on 2
Selected predictors: ['const', 'IPB51222S', 'mean_g']  AIC: -121.42475423070289
0    -120.283749
1    -119.452741
2    -119.961739
3    -120.679351
4    -121.152283
5    -119.548068
6    -120.628077
7    -120.406472
8    -120.487268
9    -119.429681
10   -121.016569
11   -120.429113
12   -121.323517
13   -120.706016
14   -119.491308
15   -119.542540
16   -120.441649
17   -119.548842
Name: AIC, dtype: float64
Processed  18 models on 3
Selected predictors: ['const', 'IPB51222S', 'mean_g', 'PERMITW']  AIC: -121.32351694314121
0    -108.369989
1    -108.887144
2    -107.827129
3    -108.692703
4    -110.824433
5    -108.1

In [65]:
ada10_text_forward

,AMZN,WMT,NVDA,TSLA,PG,NKE,AAPL,JNJ,GOOGL,MSFT,PFE,BRK-A,FB,MRK,V
R^2,0.374617,0.195994,0.440579,0.456324,0.321069,0.478031,0.518789,0.386754,0.661665,0.431262,0.265338,0.479002,0.259719,0.292207,0.564762
adj_R^2,0.317764,0.1487,0.370652,0.406899,0.236203,0.393842,0.458638,0.310098,0.619373,0.379558,0.19855,0.448355,0.216173,0.250572,0.494562
p-values,"[['const', 'CLF16OV', 'neu_g', 'HOUSTS'], [0.0...","[['const', 'UEMP27OV', 'HOUSTW'], [0.613668839...","[['const', 'AAA', 'neg_g', 'neg_s_r', 'mean_s'...","[['const', 'mean_s', 'UEMPMEAN', 'neu_s'], [8....","[['const', 'HOUSTW', 'neg_s', 'pos_s', 'neg_s_...","[['const', 'neg_s', 'UMCSENTx', 'pos_g', 'pos_...","[['const', 'mean_s', 'UEMP15OV', 'CONSPI', 'ne...","[['const', 'pos_g', 'neu_s', 'neg_s', 'IPB5122...","[['const', 'neg_g_r', 'UMCSENTx', 'VXOCLSx', '...","[['const', 'AMDMNOx', 'VXOCLSx', 'neg_s'], [4....","[['const', 'CONSPI', 'neu_g', 'IPB51222S'], [0...","[['const', 'VXOCLSx', 'UMCSENTx'], [1.94745719...","[['const', 'UMCSENTx', 'neg_g_r'], [0.00121821...","[['const', 'IPB51222S', 'mean_g'], [0.00748328...","[['const', 'PERMITW', 'VXOCLSx', 'UMCSENTx', '..."
params,"[['const', 'CLF16OV', 'neu_g', 'HOUSTS'], [4.2...","[['const', 'UEMP27OV', 'HOUSTW'], [0.035618060...","[['const', 'AAA', 'neg_g', 'neg_s_r', 'mean_s'...","[['const', 'mean_s', 'UEMPMEAN', 'neu_s'], [0....","[['const', 'HOUSTW', 'neg_s', 'pos_s', 'neg_s_...","[['const', 'neg_s', 'UMCSENTx', 'pos_g', 'pos_...","[['const', 'mean_s', 'UEMP15OV', 'CONSPI', 'ne...","[['const', 'pos_g', 'neu_s', 'neg_s', 'IPB5122...","[['const', 'neg_g_r', 'UMCSENTx', 'VXOCLSx', '...","[['const', 'AMDMNOx', 'VXOCLSx', 'neg_s'], [0....","[['const', 'CONSPI', 'neu_g', 'IPB51222S'], [0...","[['const', 'VXOCLSx', 'UMCSENTx'], [0.45127596...","[['const', 'UMCSENTx', 'neg_g_r'], [0.53192292...","[['const', 'IPB51222S', 'mean_g'], [-0.4068359...","[['const', 'PERMITW', 'VXOCLSx', 'UMCSENTx', '..."


In [66]:
ada10_text_forward.to_csv('regression 결과/ada3.csv')

### 3-1. textmining only

In [77]:
# fama
fama_text = pd.merge(textmining, fama, on = 'Date')
fama_text = fama_text.fillna(0)

for i in range(0,len(symbols)):
    fama_text_new = fama_text[fama_text['Company'] == symbols[i]]
    Y = fama_text_new[symbols[i]]
    X = fama_text_new[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',
       'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']]

    model_forward = forward_model(X, Y)
    results = model_forward
    
    if (i == 0):
        ff3_text_forward = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ff3_text_forward = pd.concat([ff3_text_forward, temp], axis = 1)
ff3_text_forward.columns = symbols

0   -69.088523
1   -73.814291
2   -72.609742
3   -74.780991
4   -68.844361
5   -70.839211
6   -69.149810
7   -69.123151
8   -69.606808
9   -68.785669
Name: AIC, dtype: float64
Processed  10 models on 1
Selected predictors: ['const', 'neu_g']  AIC: -74.7809906960023
0   -72.803840
1   -73.074356
2   -73.133791
3   -73.006890
4   -75.300940
5   -73.608338
6   -73.037930
7   -73.246310
8   -72.782423
Name: AIC, dtype: float64
Processed  9 models on 2
Selected predictors: ['const', 'neu_g', 'pos_s']  AIC: -75.30093988685769
0   -73.386990
1   -73.379965
2   -73.551153
3   -75.192242
4   -73.722799
5   -73.928263
6   -73.874502
7   -74.119439
Name: AIC, dtype: float64
Processed  8 models on 3
Selected predictors: ['const', 'neu_g', 'pos_s', 'mean_s']  AIC: -75.19224155136862
0   -105.967196
1   -106.129156
2   -105.974691
3   -106.456560
4   -108.311134
5   -108.565951
6   -105.965540
7   -106.318229
8   -106.046688
9   -108.604061
Name: AIC, dtype: float64
Processed  10 models on 1
Selecte

0   -110.216620
1   -111.801012
2   -109.926981
3   -110.353576
4   -110.831446
5   -111.693261
6   -110.028820
7   -109.758434
8   -110.175500
9   -111.999421
Name: AIC, dtype: float64
Processed  10 models on 1
Selected predictors: ['const', 'neg_s_r']  AIC: -111.9994206957368
0   -110.289776
1   -111.941087
2   -110.050084
3   -110.824499
4   -110.024218
5   -110.460849
6   -110.001397
7   -110.735268
8   -110.250644
Name: AIC, dtype: float64
Processed  9 models on 2
Selected predictors: ['const', 'neg_s_r', 'pos_g']  AIC: -111.94108693792106
0   -69.597721
1   -68.662130
2   -71.097393
3   -70.706986
4   -69.383202
5   -70.285185
6   -68.310597
7   -69.475996
8   -71.352278
9   -69.325074
Name: AIC, dtype: float64
Processed  10 models on 1
Selected predictors: ['const', 'neg_g_r']  AIC: -71.35227814963386
0   -70.921587
1   -72.351805
2   -69.599986
3   -70.709249
4   -69.679085
5   -70.687133
6   -69.352378
7   -69.701489
8   -69.714499
Name: AIC, dtype: float64
Processed  9 models

In [ ]:
ff3_text_forward.to_csv('regression 결과/textonly_forward.csv')

## Sector 별 분석

### * random forest : 5개 Sector별

In [198]:
#Create a Gaussian Classifier
from sklearn.ensemble import RandomForestRegressor

clf=RandomForestRegressor(n_estimators=100)

In [199]:
ind5_fred = pd.merge(ind5.reset_index(), fred, on = 'Date')

In [205]:
for i in range(1,6):
    #print(df_fred.columns[i])
    X = ind5_fred[ind5_fred.columns[6:]].fillna(0)
    Y = ind5_fred[ind5_fred.columns[i]]
    clf.fit(X,Y)
    feature_imp = pd.Series(clf.feature_importances_,index=ind5_fred.columns[6:]).sort_values(ascending=False)
    feature_imp = pd.DataFrame(feature_imp)
    feature_imp.columns = [ind5_fred.columns[i]]

    if (i == 1):
        random_forest_sector = feature_imp
    elif (i > 1):
        random_forest_sector = pd.concat([random_forest_sector, feature_imp], axis = 1)

In [207]:
random_forest_sector.to_csv('feature_selection/random_forest_5sector.csv')

### * Adaboost : 5개 Sector별

In [208]:
from sklearn.ensemble import AdaBoostRegressor

#Create a Gaussian Classifier
clf=AdaBoostRegressor(n_estimators=100)

for i in range(1,6):
    #print(df_fred.columns[i])
    X = ind5_fred[ind5_fred.columns[6:]].fillna(0)
    Y = ind5_fred[ind5_fred.columns[i]]
    clf.fit(X,Y)
    feature_imp = pd.Series(clf.feature_importances_,index=ind5_fred.columns[6:]).sort_values(ascending=False)
    feature_imp = pd.DataFrame(feature_imp)
    feature_imp.columns = [ind5_fred.columns[i]]

    if (i == 1):
        adaboost_sector = feature_imp
    elif (i > 1):
        adaboost_sector = pd.concat([adaboost_sector, feature_imp], axis = 1)

In [210]:
adaboost_sector.to_csv('feature_selection/adaboost_5sector.csv')

### * Sector 별 Return값

In [259]:
a = pd.DataFrame({'Industry' :['Cnsmr' for i in range(0,len(ind5))],
                  'Date':list(ind5.index),
                  'Return': list(ind5['Cnsmr'])})

b_symbols = ['Manuf','HiTec','Hlth','Other']
for i in range(0,len(b_symbols)):
    name = b_symbols[i]
    b = pd.DataFrame({'Industry' :[name for i in range(0,len(ind5))],'Date':list(ind5.index),'Return': list(ind5[b_symbols[i]])})
    a = pd.concat([a,b], axis = 0)

### 1. fama3, rf10, ada10 -> regression (5 Sector)

In [251]:
# fama
fama_sector = pd.merge(fama, ind5.reset_index(), on = 'Date')

industry_ = ['Cnsmr','Manuf','HiTec','Hlth','Other']
for i in range(0,len(industry_)):
    Y = fama_sector[industry_[i]]
    X = fama_sector[['Mkt-RF', 'SMB', 'HML']]

    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        ff3_reg_sector = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ff3_reg_sector = pd.concat([ff3_reg_sector, temp], axis = 1)
ff3_reg_sector.columns = industry_

In [253]:
ff3_reg_sector.to_csv('regression 결과/5 sector/sector_fama1.csv')

In [273]:
# rf
industry_ = ['Cnsmr','Manuf','HiTec','Hlth','Other']
for i in range(0,len(industry_)):
    new_rf = randomforest_sector[['index',industry_[i]]].sort_values(by = industry_[i], ascending = False).reset_index()
    del new_rf['level_0']
    new_rf[0:10]['index']

    col = ['Date']
    col2 = list(new_rf[0:10]['index'])
    col = col + col2

    rf_sector = pd.merge(fred[col], a, on = ['Date'])
    rf_sector = pd.merge(rf_sector, fama[['Date','RF']], on = 'Date')
    rf_sector['Return-RF'] = rf_sector['Return'] - rf_sector['RF']
    rf_sector = rf_sector.fillna(0)
    
    # 회귀분석
    rf_sector = rf_sector[rf_sector['Industry'] == industry_[i]]
    Y = rf_sector['Return-RF']
    col = col[1:]
    X = rf_sector[col]
    
    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        rf10_sector_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        rf10_sector_reg = pd.concat([rf10_sector_reg, temp], axis = 1)

rf10_sector_reg.columns = industry_    

In [275]:
rf10_sector_reg.to_csv('regression 결과/5 sector/sector_rf1.csv')

In [276]:
# ada
industry_ = ['Cnsmr','Manuf','HiTec','Hlth','Other']
for i in range(0,len(industry_)):
    new_ada = adaboost_sector[['index',industry_[i]]].sort_values(by = industry_[i], ascending = False).reset_index()
    del new_ada['level_0']
    new_ada[0:10]['index']

    col = ['Date']
    col2 = list(new_ada[0:10]['index'])
    col = col + col2

    ada_sector = pd.merge(fred[col], a, on = ['Date'])
    ada_sector = pd.merge(ada_sector, fama[['Date','RF']], on = 'Date')
    ada_sector['Return-RF'] = ada_sector['Return'] - ada_sector['RF']
    ada_sector = ada_sector.fillna(0)
    
    # 회귀분석
    ada_sector = ada_sector[ada_sector['Industry'] == industry_[i]]
    Y = ada_sector['Return-RF']
    col = col[1:]
    X = ada_sector[col]
    
    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        ada10_sector_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ada10_sector_reg = pd.concat([ada10_sector_reg, temp], axis = 1)

ada10_sector_reg.columns = industry_    

In [277]:
ada10_sector_reg.to_csv('regression 결과/5 sector/sector_ada1.csv')

### 2. fama3, rf10, ada10 + text mining -> regression (5 Sector)

In [157]:
fama_text = pd.merge(textmining, fama, on = 'Date')
fama_text = fama_text.fillna(0)

industry_mapping = {'GOOGL':'Other',
 'AMZN':'Cnsmr', 'AAPL':'HiTec', 'BRK-A':'Other',
 'FB':'HiTec', 'JNJ':'Hlth', 'MRK':'Hlth',
 'MSFT':'HiTec', 'NKE':'Manuf', 'NVDA':'Cnsmr','PG':'Manuf',
'PFE':'Hlth', 'TSLA':'Manuf', 'V':'Other', 'WMT':'Cnsmr'}

fama_text['Industry'] = fama_text.Company.map(industry_mapping)

In [158]:
a = pd.DataFrame({'Industry' :['Cnsmr' for i in range(0,len(ind5))],
                  'Date':list(ind5.index),
                  'Return': list(ind5['Cnsmr'])})

b_symbols = ['Manuf','HiTec','Hlth','Other']
for i in range(0,len(b_symbols)):
    name = b_symbols[i]
    b = pd.DataFrame({'Industry' :[name for i in range(0,len(ind5))],'Date':list(ind5.index),'Return': list(ind5[b_symbols[i]])})
    a = pd.concat([a,b], axis = 0)

#a = pd.merge(a,fama[['Date','RF']], on = 'Date')
sector_df = pd.merge(fama_text, a, on = ['Date','Industry'])

sector_df['Return-RF'] = sector_df['Return'] - sector_df['RF']

In [160]:
# fama
industry_ = ['Cnsmr','Manuf','HiTec','Hlth','Other']
for i in range(0,len(industry_)):
    sector_df_new = sector_df[sector_df['Industry'] == industry_[i]]
    Y = sector_df_new['Return-RF']
    X = sector_df_new[['Mkt-RF', 'SMB', 'HML',
       'mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',
       'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']]

    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        ff3_text_sector_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ff3_text_sector_reg = pd.concat([ff3_text_sector_reg, temp], axis = 1)
ff3_text_sector_reg.columns = industry_

In [161]:
ff3_text_sector_reg

,Cnsmr,Manuf,HiTec,Hlth,Other
R^2,0.948349,0.941956,0.955371,0.786615,0.970447
adj_R^2,0.941497,0.934257,0.949451,0.758309,0.966526
p-values,"[['Mkt-RF', 'SMB', 'HML', 'mean_g', 'pos_g', '...","[['Mkt-RF', 'SMB', 'HML', 'mean_g', 'pos_g', '...","[['Mkt-RF', 'SMB', 'HML', 'mean_g', 'pos_g', '...","[['Mkt-RF', 'SMB', 'HML', 'mean_g', 'pos_g', '...","[['Mkt-RF', 'SMB', 'HML', 'mean_g', 'pos_g', '..."
params,"[['Mkt-RF', 'SMB', 'HML', 'mean_g', 'pos_g', '...","[['Mkt-RF', 'SMB', 'HML', 'mean_g', 'pos_g', '...","[['Mkt-RF', 'SMB', 'HML', 'mean_g', 'pos_g', '...","[['Mkt-RF', 'SMB', 'HML', 'mean_g', 'pos_g', '...","[['Mkt-RF', 'SMB', 'HML', 'mean_g', 'pos_g', '..."


In [162]:
ff3_text_sector_reg.to_csv('regression 결과/ff3_text_sector.csv')

In [243]:
# rf10
randomforest_sector = pd.read_csv('feature_selection/random_forest_5sector.csv')
randomforest_sector.columns = ['index', 'Cnsmr', 'Manuf', 'HiTec', 'Hlth', 'Other']

industry_ = ['Cnsmr','Manuf','HiTec','Hlth','Other']
for i in range(0,len(industry_)):
    new_rf = randomforest_sector[['index',industry_[i]]].sort_values(by = industry_[i], ascending = False).reset_index()
    del new_rf['level_0']
    new_rf[0:10]['index']

    col = ['Date']
    col2 = list(new_rf[0:10]['index'])
    col = col + col2

    rf_text_sector = pd.merge(textmining,fred[col], on = 'Date')
    a1 = pd.merge(a,fama[['Date','RF']], on = 'Date')

    rf_text_sector = pd.merge(rf_text_sector,a1,on = 'Date')
    rf_text_sector['Return-RF'] = rf_text_sector['Return'] - rf_text_sector['RF']
    rf_text_sector = rf_text_sector.fillna(0)
    
    # 회귀분석
    rf_text_sector = rf_text_sector[rf_text_sector['Industry'] == industry_[i]]
    Y = rf_text_sector['Return-RF']
    col = col[1:] + ['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',  'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']
    X = rf_text_sector[col]
    
    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        rf10_text_sector_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        rf10_text_sector_reg = pd.concat([rf10_text_sector_reg, temp], axis = 1)

rf10_text_sector_reg.columns = industry_    

In [244]:
rf10_text_sector_reg.to_csv('regression 결과/5 sector/sector_rf2.csv')

In [240]:
# ada10
adaboost_sector = pd.read_csv('feature_selection/adaboost_5sector.csv')
adaboost_sector.columns = ['index', 'Cnsmr', 'Manuf', 'HiTec', 'Hlth', 'Other']


industry_ = ['Cnsmr','Manuf','HiTec','Hlth','Other']
for i in range(0,len(industry_)):
    new_ada = adaboost_sector[['index',industry_[i]]].sort_values(by = industry_[i], ascending = False).reset_index()
    del new_ada['level_0']
    new_ada[0:10]['index']

    col = ['Date']
    col2 = list(new_ada[0:10]['index'])
    col = col + col2

    ada_text_sector = pd.merge(textmining,fred[col], on = 'Date')
    a1 = pd.merge(a,fama[['Date','RF']], on = 'Date')

    ada_text_sector = pd.merge(ada_text_sector,a1,on = 'Date')
    ada_text_sector['Return-RF'] = ada_text_sector['Return'] - ada_text_sector['RF']
    ada_text_sector = ada_text_sector.fillna(0)
    
    # 회귀분석
    ada_text_sector = ada_text_sector[ada_text_sector['Industry'] == industry_[i]]
    Y = ada_text_sector['Return-RF']
    col = col[1:] + ['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',  'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']
    X = ada_text_sector[col]
    
    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        ada10_text_sector_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ada10_text_sector_reg = pd.concat([ada10_text_sector_reg, temp], axis = 1)

ada10_text_sector_reg.columns = industry_    

In [242]:
ada10_text_sector_reg.to_csv('regression 결과/5 sector/sector_ada2.csv')

### 3. textmining + forward selection (5 Sector)

In [283]:
# fama
industry_ = ['Cnsmr','Manuf','HiTec','Hlth','Other']
for i in range(0,len(industry_)):
    sector_df_new = sector_df[sector_df['Industry'] == industry_[i]]
    Y = sector_df_new['Return-RF']
    X = sector_df_new[['Mkt-RF', 'SMB', 'HML',
       'mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',
       'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']]

    model_forward = forward_model(X, Y)
    results = model_forward
    
    if (i == 0):
        ff3_text_sector_forward = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ff3_text_sector_forward = pd.concat([ff3_text_sector_forward, temp], axis = 1)
ff3_text_sector_forward.columns = industry_

0    -616.520054
1    -341.897985
2    -337.300372
3    -331.820283
4    -331.773705
5    -331.812622
6    -334.089464
7    -332.554161
8    -332.217733
9    -331.769049
10   -332.279738
11   -331.775145
12   -332.500834
Name: AIC, dtype: float64
Processed  13 models on 1
Selected predictors: ['const', 'Mkt-RF']  AIC: -616.5200537922072
0    -635.996656
1    -623.979050
2    -614.865991
3    -614.534401
4    -615.842947
5    -614.577382
6    -614.528768
7    -615.768327
8    -616.363229
9    -614.596426
10   -614.965102
11   -614.588022
Name: AIC, dtype: float64
Processed  12 models on 2
Selected predictors: ['const', 'Mkt-RF', 'SMB']  AIC: -635.9966560479751
0    -644.326820
1    -634.482632
2    -634.265027
3    -635.198068
4    -634.664504
5    -634.085475
6    -634.990479
7    -636.549008
8    -634.089293
9    -634.707648
10   -634.275618
Name: AIC, dtype: float64
Processed  11 models on 3
Selected predictors: ['const', 'Mkt-RF', 'SMB', 'HML']  AIC: -644.3268202901295
0   -642.6102

In [285]:
ff3_text_sector_forward.to_csv('regression 결과/5 sector/sector_fama3.csv')

In [278]:
# rf10
randomforest_sector = pd.read_csv('feature_selection/random_forest_5sector.csv')
randomforest_sector.columns = ['index', 'Cnsmr', 'Manuf', 'HiTec', 'Hlth', 'Other']


industry_ = ['Cnsmr','Manuf','HiTec','Hlth','Other']
for i in range(0,len(industry_)):
    new_rf = randomforest_sector[['index',industry_[i]]].sort_values(by = industry_[i], ascending = False).reset_index()
    del new_rf['level_0']
    new_rf[0:10]['index']

    col = ['Date']
    col2 = list(new_rf[0:10]['index'])
    col = col + col2

    rf_text_sector = pd.merge(textmining,fred[col], on = 'Date')
    a1 = pd.merge(a,fama[['Date','RF']], on = 'Date')

    rf_text_sector = pd.merge(rf_text_sector,a1,on = 'Date')
    rf_text_sector['Return-RF'] = rf_text_sector['Return'] - rf_text_sector['RF']
    rf_text_sector = rf_text_sector.fillna(0)
    
    # 회귀분석
    rf_text_sector = rf_text_sector[rf_text_sector['Industry'] == industry_[i]]
    Y = rf_text_sector['Return-RF']
    col = col[1:] + ['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',  'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']
    X = rf_text_sector[col]
    
    model_forward = forward_model(X, Y)
    results = model_forward
    
    if (i == 0):
        rf10_text_sector_forward = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        rf10_text_sector_forward = pd.concat([rf10_text_sector_forward, temp], axis = 1)

rf10_text_sector_forward.columns = industry_    

0    -1742.395073
1    -1708.303047
2    -1677.811320
3    -1819.524152
4    -1693.072235
5    -1746.168229
6    -1675.325143
7    -1751.971694
8    -1681.198880
9    -1678.133438
10   -1678.152111
11   -1676.232083
12   -1675.540417
13   -1675.955376
14   -1682.987410
15   -1677.952068
16   -1675.138222
17   -1675.040925
18   -1677.367801
19   -1680.875977
Name: AIC, dtype: float64
Processed  20 models on 1
Selected predictors: ['const', 'UMCSENTx']  AIC: -1819.5241516043216
0    -1829.795349
1    -1864.183768
2    -1843.057998
3    -2071.069915
4    -1817.631248
5    -1855.971684
6    -1820.980036
7    -1876.534261
8    -1860.993554
9    -1819.918993
10   -1819.082105
11   -1817.824030
12   -1818.029002
13   -1822.284142
14   -1817.821192
15   -1821.303386
16   -1817.782815
17   -1819.651601
18   -1820.570317
Name: AIC, dtype: float64
Processed  19 models on 2
Selected predictors: ['const', 'UMCSENTx', 'VXOCLSx']  AIC: -2071.0699154593717
0    -2080.921932
1    -2086.579930
2    -213

0    -1853.883395
1    -1848.075319
2    -1847.771962
3    -1847.824395
4    -1847.933059
5    -1847.637458
6    -1848.140021
7    -1847.608462
8    -1849.041384
9    -1847.642988
10   -1847.737390
11   -1848.413793
Name: AIC, dtype: float64
Processed  12 models on 9
Selected predictors: ['const', 'USFIRE', 'UMCSENTx', 'IPB51222S', 'UEMP15T26', 'S&P div yield', 'USGOVT', 'DNDGRG3M086SBEA', 'TWEXAFEGSMTHx', 'WPSID62']  AIC: -1853.8833945032002
0    -1852.614399
1    -1852.101595
2    -1851.988137
3    -1852.130886
4    -1851.911420
5    -1852.377761
6    -1851.967245
7    -1853.513702
8    -1851.908475
9    -1852.035939
10   -1852.654410
Name: AIC, dtype: float64
Processed  11 models on 10
Selected predictors: ['const', 'USFIRE', 'UMCSENTx', 'IPB51222S', 'UEMP15T26', 'S&P div yield', 'USGOVT', 'DNDGRG3M086SBEA', 'TWEXAFEGSMTHx', 'WPSID62', 'neg_s']  AIC: -1853.513701845577
0    -1654.908344
1    -1718.305683
2    -1631.708801
3    -1699.567974
4    -1745.916273
5    -1723.265652
6    -1

0    -2251.635406
1    -2247.937172
2    -2251.901004
3    -2250.561646
4    -2247.772167
5    -2247.965700
6    -2247.600773
7    -2247.612702
8    -2247.597237
9    -2247.946111
10   -2247.767981
11   -2247.600568
12   -2247.607590
13   -2247.631582
Name: AIC, dtype: float64
Processed  14 models on 7
Selected predictors: ['const', 'IPDCONGD', 'VXOCLSx', 'UMCSENTx', 'IPNCONGD', 'IPB51222S', 'WPSID62', 'PERMITMW']  AIC: -2251.9010035165093
0    -2256.971794
1    -2251.998709
2    -2255.139083
3    -2250.028118
4    -2250.209559
5    -2249.906319
6    -2249.928114
7    -2249.980837
8    -2250.600330
9    -2250.076389
10   -2249.914835
11   -2249.928025
12   -2250.049581
Name: AIC, dtype: float64
Processed  13 models on 8
Selected predictors: ['const', 'IPDCONGD', 'VXOCLSx', 'UMCSENTx', 'IPNCONGD', 'IPB51222S', 'WPSID62', 'PERMITMW', 'HOUSTNE']  AIC: -2256.9717935738868
0    -2256.935333
1    -2265.205549
2    -2255.147118
3    -2255.299376
4    -2254.975526
5    -2255.008597
6    -2255.

In [279]:
rf10_text_sector_forward.to_csv('regression 결과/5 sector/sector_rf3.csv')

In [280]:
# ada10
adaboost_sector = pd.read_csv('feature_selection/adaboost_5sector.csv')
adaboost_sector.columns = ['index', 'Cnsmr', 'Manuf', 'HiTec', 'Hlth', 'Other']


industry_ = ['Cnsmr','Manuf','HiTec','Hlth','Other']
for i in range(0,len(industry_)):
    new_ada = adaboost_sector[['index',industry_[i]]].sort_values(by = industry_[i], ascending = False).reset_index()
    del new_ada['level_0']
    new_ada[0:10]['index']

    col = ['Date']
    col2 = list(new_ada[0:10]['index'])
    col = col + col2

    ada_text_sector = pd.merge(textmining,fred[col], on = 'Date')
    a1 = pd.merge(a,fama[['Date','RF']], on = 'Date')

    ada_text_sector = pd.merge(ada_text_sector,a1,on = 'Date')
    ada_text_sector['Return-RF'] = ada_text_sector['Return'] - ada_text_sector['RF']
    ada_text_sector = ada_text_sector.fillna(0)
    
    # 회귀분석
    ada_text_sector = ada_text_sector[ada_text_sector['Industry'] == industry_[i]]
    Y = ada_text_sector['Return-RF']
    col = col[1:] + ['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',  'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']
    X = ada_text_sector[col]
    
    model_forward = forward_model(X, Y)
    results = model_forward
    
    if (i == 0):
        ada10_text_sector_forward = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        ada10_text_sector_forward = pd.concat([ada10_text_sector_forward, temp], axis = 1)

ada10_text_sector_forward.columns = industry_    

0    -1708.303047
1    -1819.524152
2    -1750.550351
3    -1693.072235
4    -1716.299251
5    -1733.633754
6    -1755.648144
7    -1734.232231
8    -1677.811320
9    -1678.133438
10   -1678.152111
11   -1676.232083
12   -1675.540417
13   -1675.955376
14   -1682.987410
15   -1677.952068
16   -1675.138222
17   -1675.040925
18   -1677.367801
19   -1680.875977
Name: AIC, dtype: float64
Processed  20 models on 1
Selected predictors: ['const', 'UMCSENTx']  AIC: -1819.5241516043216
0    -1864.183768
1    -1862.616460
2    -2071.069915
3    -1853.049066
4    -1819.232754
5    -1831.912380
6    -1837.839251
7    -1843.057998
8    -1860.993554
9    -1819.918993
10   -1819.082105
11   -1817.824030
12   -1818.029002
13   -1822.284142
14   -1817.821192
15   -1821.303386
16   -1817.782815
17   -1819.651601
18   -1820.570317
Name: AIC, dtype: float64
Processed  19 models on 2
Selected predictors: ['const', 'UMCSENTx', 'VXOCLSx']  AIC: -2071.0699154593717
0    -2086.579930
1    -2069.385982
2    -216

0    -1894.981616
1    -1885.949572
2    -1885.081607
3    -1885.097907
4    -1884.805340
5    -1885.800535
6    -1884.821041
7    -1885.256947
8    -1885.119802
9    -1885.999434
10   -1886.188014
Name: AIC, dtype: float64
Processed  11 models on 10
Selected predictors: ['const', 'USFIRE', 'COMPAPFFx', 'UMCSENTx', 'USGOVT', 'PERMITS', 'CLAIMSx', 'CUSR0000SAC', 'DNDGRG3M086SBEA', 'S&P div yield', 'PERMIT']  AIC: -1894.9816161767071
0   -1894.165337
1   -1893.197201
2   -1893.299886
3   -1892.985314
4   -1894.529726
5   -1892.981839
6   -1893.709409
7   -1893.213683
8   -1894.125050
9   -1894.910891
Name: AIC, dtype: float64
Processed  10 models on 11
Selected predictors: ['const', 'USFIRE', 'COMPAPFFx', 'UMCSENTx', 'USGOVT', 'PERMITS', 'CLAIMSx', 'CUSR0000SAC', 'DNDGRG3M086SBEA', 'S&P div yield', 'PERMIT', 'neg_s_r']  AIC: -1894.910890678787
0    -1654.908344
1    -1745.916273
2    -1656.044520
3    -1723.265652
4    -1635.315895
5    -1687.293298
6    -1633.959776
7    -1708.233494
8 

0    -2125.020015
1    -2126.878218
2    -2138.618466
3    -2125.003049
4    -2125.091651
5    -2124.858176
6    -2125.369288
7    -2125.279761
8    -2124.816217
9    -2124.807493
10   -2124.809596
11   -2124.807280
12   -2124.892040
13   -2124.925943
14   -2125.059569
Name: AIC, dtype: float64
Processed  15 models on 6
Selected predictors: ['const', 'IPDCONGD', 'VXOCLSx', 'UMCSENTx', 'IPB51222S', 'HOUSTW', 'COMPAPFFx']  AIC: -2138.6184663025274
0    -2153.720456
1    -2137.234004
2    -2137.082552
3    -2150.028289
4    -2136.659018
5    -2137.469654
6    -2137.541073
7    -2136.646201
8    -2136.619026
9    -2136.692285
10   -2136.680470
11   -2136.758251
12   -2136.775367
13   -2136.981493
Name: AIC, dtype: float64
Processed  14 models on 7
Selected predictors: ['const', 'IPDCONGD', 'VXOCLSx', 'UMCSENTx', 'IPB51222S', 'HOUSTW', 'COMPAPFFx', 'WPSID62']  AIC: -2153.7204563528435
0    -2154.751800
1    -2156.645752
2    -2160.189533
3    -2151.751684
4    -2152.177370
5    -2152.330073

In [282]:
ada10_text_sector_forward.to_csv('regression 결과/5 sector/sector_ada3.csv')

### 3-1. Textmining Only (5 Sector)

In [309]:
# regression
ind_text = sector_df[['Company', 'Date', 'mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',
       'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r', 'Industry', 'Return-RF']].fillna(0)

industry_ = ['Cnsmr','Manuf','HiTec','Hlth','Other']
for i in range(0,len(industry_)):
    ind_text_new = ind_text[ind_text['Industry'] == industry_[i]]
    Y = ind_text_new['Return-RF']
    X = ind_text_new[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',
       'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']]

    model = sm.OLS(Y, X)
    results = model.fit()
    
    if (i == 0):
        text_only_reg = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        text_only_reg = pd.concat([text_only_reg, temp], axis = 1)
text_only_reg.columns = industry_

In [311]:
text_only_reg.to_csv('regression 결과/5 sector/textonly_regression.csv')

In [306]:
# forward
ind_text = sector_df[['Company', 'Date', 'mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',
       'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r', 'Industry', 'Return-RF']].fillna(0)

industry_ = ['Cnsmr','Manuf','HiTec','Hlth','Other']
for i in range(0,len(industry_)):
    ind_text_new = ind_text[ind_text['Industry'] == industry_[i]]
    Y = ind_text_new['Return-RF']
    X = ind_text_new[['mean_g', 'pos_g', 'neg_g', 'neu_g', 'mean_s',
       'pos_s', 'neg_s', 'neu_s', 'neg_g_r', 'neg_s_r']]

    model_forward = forward_model(X, Y)
    results = model_forward
    
    if (i == 0):
        text_only_forward = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
    else:
        temp = pd.DataFrame({'R^2': [results.rsquared],
              'adj_R^2' :[results.rsquared_adj], 
              'p-values': str([list(dict(results.pvalues)),list(results.pvalues)]),
              'params': str([list(dict(results.params)),list(results.params)])  
             }).T
        text_only_forward = pd.concat([text_only_forward, temp], axis = 1)
text_only_forward.columns = symbols

0   -331.820283
1   -331.773705
2   -331.812622
3   -334.089464
4   -332.554161
5   -332.217733
6   -331.769049
7   -332.279738
8   -331.775145
9   -332.500834
Name: AIC, dtype: float64
Processed  10 models on 1
Selected predictors: ['const', 'neu_g']  AIC: -334.0894638887761
0   -332.090021
1   -332.932634
2   -332.163424
3   -332.911420
4   -332.422427
5   -332.098240
6   -332.735084
7   -332.175838
8   -332.814565
Name: AIC, dtype: float64
Processed  9 models on 2
Selected predictors: ['const', 'neu_g', 'pos_g']  AIC: -332.93263406797837
0   -314.747357
1   -315.034841
2   -314.680592
3   -314.811937
4   -318.618291
5   -314.787686
6   -316.813153
7   -314.925220
8   -314.685778
9   -316.535772
Name: AIC, dtype: float64
Processed  10 models on 1
Selected predictors: ['const', 'mean_s']  AIC: -318.61829059790557
0   -316.640741
1   -316.855475
2   -316.841081
3   -316.795843
4   -317.345856
5   -317.494566
6   -316.771023
7   -316.887466
8   -316.748981
Name: AIC, dtype: float64
Proc

ValueError: Length mismatch: Expected axis has 5 elements, new values have 15 elements

In [307]:
text_only_forward.to_csv('regression 결과/5 sector/textonly_forward.csv')